<div class="alert alert-info">
<h3>Homework</h3>
<p>The file winereviews.csv has more than 85,000 user submitted wine reviews. What are the most common words used in reviews (located in the "description" field)? What is the sentiment in the reviews? Bonus: Is there variation in the words used for reviews above and below the median review score? Super bonus: Could you construct a wine sentiment dictionary? 

<p> https://github.com/nealcaren/KULeuvenBigData/blob/master/notebooks/data/wine_reviews.csv?raw=true
</div>





    

Import the most likely libraries.

In [1]:
%matplotlib inline

import pandas as pd
import seaborn as sns

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from afinn import Afinn

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


Load the data from the internet. Try to only do this once because it takes a couple of minutes, even on a fast internet connect.

In [2]:
wine_url = 'https://github.com/nealcaren/KULeuvenBigData/blob/master/notebooks/data/wine_reviews.csv?raw=true'
wine_df = pd.read_csv(wine_url)

Save the data, so that the next time, we can just load it locally.

In [3]:
wine_df.to_csv('data/wine_reviews.csv')

In [4]:
wine_df = pd.read_csv('data/wine_reviews.csv')

Inspection

In [5]:
wine_df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,rating
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,Low
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,Low
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Low
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,Low
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Low


In [6]:
wine_df.describe()

,Unnamed: 0,points,price
count,85128.000000,85128.000000,79276.000000
mean,42563.500000,88.177016,36.888579
std,24574.481195,3.676081,46.700137
min,0.000000,80.000000,4.000000
25%,21281.750000,85.000000,15.000000
50%,42563.500000,87.000000,25.000000
75%,63845.250000,92.000000,45.000000
max,85127.000000,100.000000,2500.000000


In [7]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85128 entries, 0 to 85127
Data columns (total 15 columns):
Unnamed: 0               85128 non-null int64
country                  85099 non-null object
description              85128 non-null object
designation              60343 non-null object
points                   85128 non-null int64
price                    79276 non-null float64
province                 85099 non-null object
region_1                 71476 non-null object
region_2                 33529 non-null object
taster_name              65765 non-null object
taster_twitter_handle    62785 non-null object
title                    85128 non-null object
variety                  85128 non-null object
winery                   85128 non-null object
rating                   85128 non-null object
dtypes: float64(1), int64(2), object(12)
memory usage: 9.7+ MB


What are the most common words used in reviews (located in the "description" field)?

In [8]:
vectorizer = CountVectorizer(max_features=1000)

In [9]:
vectorizer.fit(wine_df['description'])
len(vectorizer.get_feature_names())

1000

In [10]:
frequency_array = vectorizer.transform(wine_df['description'])

In [11]:
word_freq_df = pd.DataFrame(frequency_array.toarray(),
                            columns = vectorizer.get_feature_names())

In [12]:
word_freq_df.sum().sort_values(ascending=False)[:20]

and        225810
the        143854
of         110486
with        79130
this        74461
is          64605
it          57057
wine        52801
flavors     41615
in          40839
to          36730
fruit       31912
on          29717
that        27027
aromas      24957
palate      24094
finish      22475
acidity     22435
but         21455
from        21171
dtype: int64

In [13]:
wine_df.describe()

,Unnamed: 0,points,price
count,85128.000000,85128.000000,79276.000000
mean,42563.500000,88.177016,36.888579
std,24574.481195,3.676081,46.700137
min,0.000000,80.000000,4.000000
25%,21281.750000,85.000000,15.000000
50%,42563.500000,87.000000,25.000000
75%,63845.250000,92.000000,45.000000
max,85127.000000,100.000000,2500.000000


In [14]:
wine_df['High Score'] = wine_df['points'] > 87

In [15]:
wine_df['High Score'].mean()

0.3951108918334743

In [16]:
vectorizer2 = CountVectorizer(max_features=500, 
                             stop_words='english',
                              binary=True)

In [17]:


vectorizer2.fit(wine_df['description'])

frequency_array = vectorizer2.transform(wine_df['description'])

word_freq_df = pd.DataFrame(frequency_array.toarray(),
                            columns = vectorizer2.get_feature_names())

In [18]:
word_freq_df.head()

,10,100,20,2015,2016,2017,2018,2019,2020,2022,...,wood,woody,year,years,yeasty,yellow,young,zest,zesty,zinfandel
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
word_freq_df[wine_df['High Score'] == True].sum().sort_values(ascending=False)[:20]

wine       19012
flavors    13765
fruit      12411
drink      10123
palate     10050
tannins     9644
black       9152
acidity     8967
aromas      8443
ripe        8230
finish      8139
cherry      7844
rich        6929
spice       5916
notes       4861
dark        4785
red         4666
nose        4646
oak         4547
fruits      4100
dtype: int64

In [20]:
word_freq_df[wine_df['High Score'] == False].sum().sort_values(ascending=False)[:20]

flavors    25898
wine       23468
aromas     16201
fruit      16118
finish     14181
palate     13488
acidity    12918
drink       9609
cherry      9500
tannins     8943
red         7851
ripe        7480
dry         7142
soft        6942
fresh       6774
notes       6530
berry       6374
sweet       6313
nose        6217
light       6205
dtype: int64

Sentiment!

In [21]:
afinn = Afinn(language='en')

In [22]:
wine_df['Afinn Score'] = wine_df['description'].apply(afinn.score)

In [23]:
def word_count(text_string):
    '''Calculate the number of words in a string'''
    return len(text_string.split())

In [24]:
wine_df['Word Count'] = wine_df['description'].apply(word_count)

In [25]:
wine_df['Afinn Score Adjusted'] = wine_df['Afinn Score' ] / wine_df['Word Count'] 

In [26]:
new_columns = ['Afinn Score', 'Word Count', 'Afinn Score Adjusted']

In [27]:
wine_df[new_columns].describe()

,Afinn Score,Word Count,Afinn Score Adjusted
count,85128.00000,85128.000000,85128.000000
mean,3.50128,40.090816,0.086331
std,3.52458,11.908889,0.085814
min,-9.00000,5.000000,-0.583333
25%,1.00000,32.000000,0.025000
50%,3.00000,39.000000,0.076923
75%,5.00000,47.000000,0.136364
max,29.00000,135.000000,0.689655


In [28]:
wine_df.groupby('High Score')[new_columns].median()

,Afinn Score,Word Count,Afinn Score Adjusted
High Score,,,
False,2.0,34,0.062500
True,5.0,47,0.096154


In [29]:
wine_df.groupby('High Score')[new_columns].mean()

,Afinn Score,Word Count,Afinn Score Adjusted
High Score,,,
False,2.531004,34.913095,0.073734
True,4.986710,48.017571,0.105617


In [30]:
vader_analyzer = SentimentIntensityAnalyzer()

In [31]:
vader_sentiment = wine_df['description'].apply(vader_analyzer.polarity_scores)

In [32]:
vader_sentiment

0        {'neg': 0.0, 'neu': 0.935, 'pos': 0.065, 'comp...
1        {'neg': 0.0, 'neu': 0.868, 'pos': 0.132, 'comp...
2        {'neg': 0.053, 'neu': 0.947, 'pos': 0.0, 'comp...
3        {'neg': 0.0, 'neu': 0.926, 'pos': 0.074, 'comp...
4        {'neg': 0.0, 'neu': 0.805, 'pos': 0.195, 'comp...
5        {'neg': 0.0, 'neu': 0.96, 'pos': 0.04, 'compou...
6        {'neg': 0.0, 'neu': 0.843, 'pos': 0.157, 'comp...
7        {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
8        {'neg': 0.0, 'neu': 0.586, 'pos': 0.414, 'comp...
9        {'neg': 0.0, 'neu': 0.808, 'pos': 0.192, 'comp...
10       {'neg': 0.0, 'neu': 0.79, 'pos': 0.21, 'compou...
11       {'neg': 0.0, 'neu': 0.865, 'pos': 0.135, 'comp...
12       {'neg': 0.0, 'neu': 0.878, 'pos': 0.122, 'comp...
13       {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
14       {'neg': 0.032, 'neu': 0.923, 'pos': 0.045, 'co...
15       {'neg': 0.0, 'neu': 0.802, 'pos': 0.198, 'comp...
16       {'neg': 0.0, 'neu': 0.937, 'pos': 0.063, 'comp.

In [33]:
vader_df = pd.DataFrame(vader_sentiment.tolist())

In [34]:
vader_df.head()

,compound,neg,neu,pos
0,0.1531,0.000,0.935,0.065
1,0.6486,0.000,0.868,0.132
2,-0.1280,0.053,0.947,0.000
3,0.3400,0.000,0.926,0.074
4,0.8176,0.000,0.805,0.195


In [35]:
vader_df['High Score'] = wine_df['High Score']

In [36]:
vader_df.head()

,compound,neg,neu,pos,High Score
0,0.1531,0.000,0.935,0.065,False
1,0.6486,0.000,0.868,0.132,False
2,-0.1280,0.053,0.947,0.000,False
3,0.3400,0.000,0.926,0.074,False
4,0.8176,0.000,0.805,0.195,False


In [37]:
vader_df.groupby('High Score').mean()

,compound,neg,neu,pos
High Score,,,,
False,0.404178,0.020944,0.854898,0.124157
True,0.642529,0.011839,0.835308,0.152852


In [38]:
vader_df.groupby('High Score').median()

,compound,neg,neu,pos
High Score,,,,
False,0.4767,0.0,0.862,0.113
True,0.7579,0.0,0.839,0.148


In [39]:
vader_df['Afinn Score Adjusted'] = wine_df['Afinn Score Adjusted']

In [40]:
vader_df.groupby('High Score').median()

,compound,neg,neu,pos,Afinn Score Adjusted
High Score,,,,,
False,0.4767,0.0,0.862,0.113,0.062500
True,0.7579,0.0,0.839,0.148,0.096154


In [41]:
from sklearn import linear_model

In [42]:
reg = linear_model.LinearRegression(normalize=True)

In [43]:
x_vars = ['Afinn Score Adjusted', 'compound']

In [44]:
reg.fit(vader_df[x_vars], vader_df['High Score'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [45]:
reg.coef_

array([-0.33011586,  0.43262596])

Now, to create our own sentiment list.

In [46]:
word_freq_df[wine_df['High Score'] == True].sum()

10               726
100              816
20               377
2015             565
2016             758
2017            1256
2018            1991
2019            1038
2020            1880
2022             896
2025             844
accented         449
accents          488
acid             283
acidic           199
acidity         8967
acids            463
add              524
adds             552
aftertaste       497
age             2953
aged            1004
aging           2348
alcohol          727
almond           527
alongside       1227
ample            446
anise           1076
appealing        292
appellation      579
               ...  
velvety         1254
verdot           760
vibrant          837
vines            883
vineyard        3275
vineyards        981
vintage         1870
viognier         283
violet           831
warm             579
way              721
weight          1099
wet              502
whiff            629
white           2717
wild            1468
wine         

In [47]:
high_words = word_freq_df[wine_df['High Score'] == True].sum()

In [48]:
word_freq_df[wine_df['High Score'] == False].sum()

10               437
100              511
20               365
2015             276
2016             546
2017             538
2018             448
2019             201
2020              99
2022              10
2025               4
accented         541
accents          515
acid             558
acidic           895
acidity        12918
acids            886
add              516
adds             510
aftertaste      1379
age              908
aged             710
aging           1012
alcohol         1528
almond          1198
alongside       1636
ample            353
anise            559
appealing        398
appellation      235
               ...  
velvety          179
verdot           425
vibrant          569
vines            292
vineyard         798
vineyards        350
vintage         1008
viognier         519
violet           377
warm            1073
way              963
weight          1228
wet              493
whiff            696
white           4367
wild            1039
wine         

In [49]:
low_words = word_freq_df[wine_df['High Score'] == False].sum()

In [50]:
word_df = pd.concat([low_words, high_words], axis=1)

In [51]:
word_df.head()

,0,1
10,437,726
100,511,816
20,365,377
2015,276,565
2016,546,758


In [52]:
word_df['ratio'] = word_df[1] /  word_df[0] * 100

In [53]:
word_df.sort_values(by='ratio').head(10)

,0,1,ratio
easygoing,692,11,1.589595
simple,3428,56,1.633606
straightforward,1047,34,3.247373
stalky,636,21,3.301887
lacks,682,25,3.665689
basic,761,37,4.862024
flat,679,42,6.185567
short,1023,79,7.722385
easy,3183,290,9.110902
rubbery,687,67,9.752547


In [54]:
word_df.sort_values(by='ratio', ascending = False).head(10)

,0,1,ratio
2025,4,844,21100.000000
2022,10,896,8960.000000
2020,99,1880,1898.989899
beautifully,74,996,1345.945946
beautiful,89,1113,1250.561798
impressive,115,1326,1153.043478
refined,73,686,939.726027
velvety,179,1254,700.558659
opulent,124,807,650.806452
lovely,184,1120,608.695652


In [55]:
word_df[0].sum()

739297

In [56]:
word_df[1].sum()

626901

In [57]:
word_df['low percent'] = word_df[0] / word_df[0].sum() * 100

In [58]:
word_df['high percent'] = word_df[1] / word_df[1].sum() * 100

In [59]:
word_df['ratio'] = word_df['low percent'] /  word_df['high percent'] * 100

In [60]:
word_df.sort_values(by='ratio', ascending = False).head(10)

,0,1,ratio,low percent,high percent
easygoing,692,11,5334.496420,0.093602,0.001755
simple,3428,56,5190.782179,0.463684,0.008933
straightforward,1047,34,2611.245974,0.141621,0.005424
stalky,636,21,2568.134941,0.086028,0.003350
lacks,682,25,2313.259662,0.092250,0.003988
basic,761,37,1744.066143,0.102936,0.005902
flat,679,42,1370.883353,0.091844,0.006700
short,1023,79,1098.066295,0.138375,0.012602
easy,3183,290,930.719171,0.430544,0.046259
rubbery,687,67,869.484718,0.092926,0.010687


In [61]:
word_df.sort_values(by='ratio', ascending = False).tail(10)

,0,1,ratio,low percent,high percent
lovely,184,1120,13.930921,0.024889,0.178657
opulent,124,807,13.029513,0.016773,0.128728
velvety,179,1254,12.104184,0.024212,0.200032
refined,73,686,9.023578,0.009874,0.109427
impressive,115,1326,7.354181,0.015555,0.211517
beautiful,89,1113,6.780705,0.012038,0.177540
beautifully,74,996,6.300172,0.010010,0.158877
2020,99,1880,4.465369,0.013391,0.299888
2022,10,896,0.946394,0.001353,0.142925
2025,4,844,0.401881,0.000541,0.134631


In [62]:
word_df['ratio'].describe()

count     500.000000
mean      169.456840
std       402.875365
min         0.401881
25%        55.146568
50%        97.000888
75%       155.919509
max      5334.496420
Name: ratio, dtype: float64

In [63]:
word_df['ratio'].quantile(q=.25)

55.14656823657506

In [64]:
word_df[word_df['ratio'] < word_df['ratio'].quantile(q=.25)].index

Index(['10', '100', '2015', '2017', '2018', '2019', '2020', '2022', '2025',
       'age',
       ...
       'verdot', 'vines', 'vineyard', 'vineyards', 'vintage', 'violet',
       'winemaker', 'winery', 'wines', 'years'],
      dtype='object', length=125)

In [65]:
word_df[word_df['ratio'] > word_df['ratio'].quantile(q=.75) ].index

Index(['acid', 'acidic', 'acids', 'aftertaste', 'alcohol', 'almond', 'apple',
       'aroma', 'aromas', 'astringent',
       ...
       'tastes', 'tasting', 'tomato', 'touch', 'tough', 'tropical', 'value',
       'warm', 'woody', 'zinfandel'],
      dtype='object', length=125)